# UWB 자율주행 물류 로봇 통합 관리 시스템

### DB 접속, 해제

In [1]:
import cx_Oracle
import os

# DB 접속|
def conn_db() :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기    
    conn = cx_Oracle.connect('dct','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    return conn, cursor

# DB 접속 해제
def close_db(conn, cursor) :
    cursor.close()
    conn.close()

## 회원가입

In [2]:
# 아이디 중복 체크
def check_id(M_ID) :
    conn, cursor = conn_db()  # DB 접속
    
    query = f"""SELECT COUNT(*) FROM MEMBERS WHERE M_ID = :1"""
    cursor.execute(query, [M_ID])  # 쿼리문 실행 
    data = cursor.fetchone()  # 조회 결과(한 튜플) 가져오기
    close_db(conn, cursor)  # DB 접속 해제
    
    if data[0] == 0:
        return jsonify({'message':'success', 'result': True})
    else :
        return jsonify({'message':'success', 'result': False})
    
# 회원가입
def signup_member(M_NAME, M_ID, M_PW, M_TEL) :
    try :
        conn, cursor = conn_db()  # DB 접속

        query = f"""INSERT INTO MEMBERS (M_NAME, M_ID, M_PW, M_TEL) VALUES (:1, :2, :3, :4)"""
        cursor.execute(query, [M_NAME, M_ID, M_PW, M_TEL])  # 쿼리문 실행 
        conn.commit()
    
        return jsonify({'message':'success', 'result': True})
    except :
        return False

## 로그인

In [3]:
# 로그인
def login_member(M_ID, M_PW) :
    conn, cursor = conn_db()  # DB 접속
    
    query = f"""SELECT M_NO, M_NAME, M_ID FROM MEMBERS WHERE M_ID = :1 AND M_PW = :2"""
    cursor.execute(query, [M_ID, M_PW])  # 쿼리문 실행 
    data = cursor.fetchone()  # 조회 결과(한 튜플) 가져오기
    close_db(conn, cursor)  # DB 접속 해제
    
    if data != None:
        session['user_no'] = data[0]
        session['user_name'] = data[1]
        session['user_id'] = data[2]
        return jsonify({'message':'success', 'result': True})
    else :
        return jsonify({'message':'success', 'result': False})

# 관리 및 설정

### 설계도

In [4]:
from datetime import datetime

UPLOAD_FOLDER = './static/upload'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

# setting html에 출력할 정보 딕셔너리
set_list = {
    'area': [], 
    'fixed_tag': [],
    'robot': [],
    'user_tag': [],
    'way': []
    }


# 업로드 파일 확장자 확인
def allowed_file(filename):  
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS and filename not in os.listdir(UPLOAD_FOLDER)

# 업로드 파일명 설정
def set_filename(filename) :  
    ext = filename.rsplit('.', 1)[1]
    now = datetime.now().strftime('%Y%m%d%H%M%S')
    return f"{now}.{ext}"
    
# 공간 등록
def areaNameCk(M_NO, A_NAME, filename) :
    try :
        conn, cursor = conn_db()  # DB 접속
        
        A_SRC = set_filename(filename)
        query = f"""INSERT INTO AREA (M_NO, A_NAME, A_SRC) VALUES (:1, :2, :3)"""
        cursor.execute(query, [M_NO, A_NAME, A_SRC])  # 쿼리문 실행 
        conn.commit()
        
        query = f"""SELECT A_NO FROM AREA WHERE A_SRC = :1"""
        cursor.execute(query, [A_SRC])  # 쿼리문 실행 
        a_no = cursor.fetchone()  # 조회 결과(한 튜플) 가져오기
        close_db(conn, cursor)  # DB 접속 해제
        
        return [A_SRC, a_no]
    except :
        return False
 
# # 고정형 태그 모두 조회 by 공간 번호
# def allFixedTag(A_NO) :
#     conn, cursor = conn_db()  # DB 접속
    
#     query = f"""SELECT FT_NO, FT_ID, FT_STATE, FT_X, FT_Y FROM AREA WHERE A_NO = :1"""
#     cursor.execute(query, [A_NO])  # 쿼리문 실행 
#     data = cursor.fetchall()  # 조회 결과(한 튜플) 가져오기
#     close_db(conn, cursor)  # DB 접속 해제
    
#     return data

# # 로봇 모두 조회 by 공간 번호
# def allRobot(A_NO) :
#     conn, cursor = conn_db()  # DB 접속
    
#     query = f"""SELECT R_NO, R_MODEL, R_ANCHOR1_ID, R_ANCHOR2_ID, R_ANCHOR3_ID FROM AREA WHERE A_NO = :1"""
#     cursor.execute(query, [A_NO])  # 쿼리문 실행 
#     data = cursor.fetchall()  # 조회 결과(한 튜플) 가져오기
#     close_db(conn, cursor)  # DB 접속 해제
    
#     return data

# # 사용자 태그 모두 조회 by 공간 번호
# def allUserTag(A_NO) :
#     conn, cursor = conn_db()  # DB 접속
    
#     query = f"""SELECT UT_NO, UT_ID FROM AREA WHERE A_NO = :1"""
#     cursor.execute(query, [A_NO])  # 쿼리문 실행 
#     data = cursor.fetchall()  # 조회 결과(한 튜플) 가져오기
#     close_db(conn, cursor)  # DB 접속 해제
    
#     return data

# set_list 정보 조회 by 공간 번호
def selectByANO(A_NO) :
    A_NO = int(A_NO)
    
    conn, cursor = conn_db()  # DB 접속
    
    query = f"""SELECT A_NO, A_NAME, A_SRC FROM AREA WHERE M_NO = :1 ORDER BY A_NO"""
    cursor.execute(query, [m_no])  # 쿼리문 실행 
    area = cursor.fetchall()  # 조회 결과(한 튜플) 가져오기
    set_list['area'] = area
    
    if area != [] :
        query = f"""SELECT FT_NO, FT_ID, FT_STATE, FT_X, FT_Y FROM FIXED_TAG WHERE A_NO = :1 ORDER BY FT_NO"""
        cursor.execute(query, [A_NO])  # 쿼리문 실행 
        fixed_tag = cursor.fetchall()  # 조회 결과(한 튜플) 가져오기
        set_list['fixed_tag'] = fixed_tag

        query = f"""SELECT R_NO, R_MODEL, R_STATE, R_ANCHOR1_ID, R_ANCHOR1_STATE, R_ANCHOR2_ID, R_ANCHOR2_STATE,R_ANCHOR3_ID, R_ANCHOR3_STATE, UT_NO, W_NO FROM ROBOT WHERE A_NO = :1 ORDER BY R_NO"""
        cursor.execute(query, [A_NO])  # 쿼리문 실행 
        robot = cursor.fetchall()  # 조회 결과(한 튜플) 가져오기
        set_list['robot'] = robot

        query = f"""SELECT UT_NO, UT_ID, UT_STATE FROM USER_TAG WHERE A_NO = :1 ORDER BY UT_NO"""
        cursor.execute(query, [A_NO])  # 쿼리문 실행 
        user_tag = cursor.fetchall()  # 조회 결과(한 튜플) 가져오기
        set_list['user_tag'] = user_tag

        query = f"""SELECT W_NO, W_NAME, W_X, W_Y FROM WAY WHERE A_NO = :1 ORDER BY W_NO"""
        cursor.execute(query, [A_NO])  # 쿼리문 실행 
        way = cursor.fetchall()  # 조회 결과(한 튜플) 가져오기
        set_list['way'] = way 

    close_db(conn, cursor)  # DB 접속 해제


##### 데이터 추가 후, session['areas'] 갱신하기!

# 홈

In [5]:
# 모든 공간 조회 by 회원 번호
def allAreas(m_no) :
    conn, cursor = conn_db()  # DB 접속
    
    query = f"""SELECT A_NO, A_NAME, A_SRC FROM AREA WHERE M_NO = :1 ORDER BY A_NO"""
    cursor.execute(query, [m_no])  # 쿼리문 실행 
    area = cursor.fetchall()  # 조회 결과(한 튜플) 가져오기
    set_list['area'] = area
    close_db(conn, cursor)  # DB 접속 해제
    

# 전체 공간 정보 리스트에서 원하는 공간 정보 가져오기 by 공간 번호
def selectArea(areas, a_no) :
    a_no = int(a_no)
    for area in areas :
        if area[0] == a_no :
            return area
    return []

In [ ]:
from flask import Flask, session, render_template, redirect, request, url_for, jsonify
from werkzeug.utils import secure_filename
from markupsafe import escape
from datetime import timedelta, datetime
import random

app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=10) # 로그인 지속 시간
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


############# 회원 가입 #############
@app.route('/signup')  # 회원 가입
def signup() :
    if len(session) :
        return redirect(url_for('home'))
    else :
        return render_template('signup.html', title="회원 가입")


@app.route('/signup/idck' , methods=["POST"])  # 아이디 중복 체크
def ck_id() :
    if request.method == 'POST' :
        M_ID = request.get_json()['M_ID']
        return check_id(M_ID)
    else :
        return rendirect(url_for('internal_server_error', e=500))

@app.route('/signup/result', methods=["POST"])  # 회원 가입 결과
def signup_result() :
    if request.method == 'POST' :
        M_NAME = request.form['M_NAME']
        M_ID = request.form['M_ID']
        M_PW = request.form['M_PW']
        M_TEL = request.form['M_TEL']
        return signup_member(M_NAME, M_ID, M_PW, M_TEL)
    else :
        return rendirect(url_for('internal_server_error', e=500))
    
######### 로그인 #########
@app.route('/') # 로그인 
def login() :
    if len(session) :
        return redirect(url_for('home'))
    else :
        return render_template('login.html', title="로그인")


@app.route('/login/result', methods=['POST']) # 로그인 + 입력값 체크
def login_result() :
    if request.method == 'POST' :
        M_ID = request.form['M_ID']
        M_PW = request.form['M_PW']
        return login_member(M_ID, M_PW)
    else :
        return redirect(url_for('internal_server_error', e=500))
    
######### 로그아웃 #########
@app.route('/logout') # 로그아웃
def logout() :
    if len(session):
        session.clear('user_no', None)
        session.clear('user_name', None)
        session.clear('user_id', None)
        return redirect(url_for('login'))
    else :
        return redirect(url_for('internal_server_error', e=500))


######### 홈 #########
@app.route('/home')  # 홈
def home() :
    if len(session):
        allAreas(session['user_no'])
        
        if set_list['area'] != [] :
            return redirect(url_for('showA_NO', a_no=set_list['area'][0][0]))
        else :
            return render_template('home.html', title="홈")
    else :
        return redirect(url_for('login'))
    
@app.route('/home/<a_no>')  # 홈 when 공간번호 = a_no
def showA_NO(a_no) :
    if len(session):
        a_info = selectArea(set_list['area'], a_no)
        
        if a_info != []:
            return render_template('home.html', title="홈", set_list=set_list, a_info=a_info)
        else :
            return redirect(url_for('page_not_found', e=404))
    else :
        return redirect(url_for('login'))


######### 관리 및 설정 #########
@app.route('/set')  # 관리 및 설정
def setting() :
    if len(session) :
        allAreas(session['user_no'])
        
        return render_template('setting.html', title="관리 및 설정", set_list=set_list)
    else :
        return redirect(url_for('login'))
    
@app.route('/set/<a_no>')  # 관리 및 설정 when 공간번호 = a_no
def setA_NO(a_no) :
    print(a_no)
    if len(session) :
        if a_no == "new" :
            return render_template('setting.html', title="관리 및 설정", set_list=set_list, a_info=[])
        else :
            a_info = selectArea(set_list['area'], a_no)

            if a_info != [] :
                return render_template('setting.html', title="관리 및 설정", set_list=set_list, a_info=a_info)
            else :
                return redirect(url_for('page_not_found', e=404))
    else :
        return redirect(url_for('login'))


@app.route('/set/new/upload_result', methods=['POST'])  # 설계도면 이미지 업로드
def upload_file():
    if len(session) :
        if request.method == 'POST':
            a_name = request.form['A_NAME']
            file = request.files['file']
            angle = request.form['angle']
            
            if file.filename == '':
                return jsonify({'message':'파일을 선택해 주십시오.', 'result': False})

            if file and allowed_file(file.filename):
                filename = secure_filename(set_filename(file.filename))
                file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
                result = areaNameCk(session['user_no'], a_name, filename)

                if result != False:
                    allAreas(session['user_no'])
                    return jsonify({'message':'농장 설계도가 등록되었습니다.', 'result': True, 'fname': result[0], 'a_no':result[1]})
                else :
                    return jsonify({'message':'농장 설계도 등록 실패', 'result': False})
            else :
                return jsonify({'message':'이미지 파일을 선택해 주십시오.(확장자: jpg, png, jpeg)', 'result': False})

    else :
        return jsonify({'message':'로그인이 되어있지 않습니다.', 'result': False})

######### 에러 처리 #########
@app.errorhandler(404)  # 존재하지 않는 페이지 에러
def page_not_found(e):
    return render_template("404.html"), 404    
    
@app.errorhandler(500)  # 서버 에러
def internal_server_error(e):
    return render_template("500.html"), 500


if __name__ == '__main__' :
    app.run(host="192.168.70.205", port=2048)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.70.205:2048
Press CTRL+C to quit
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET / HTTP/1.1" 200 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/bootstrap.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/common.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/footer.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/common.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/prism.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/sweetalert.min.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/jquery.canvasResize.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:39] "GET /static/canvasResize.js HTTP/1.1" 304 -
192.168.7

new


192.168.70.205 - - [13/Dec/2022 21:34:58] "POST /set/new/upload_result HTTP/1.1" 200 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /set/70 HTTP/1.1" 200 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/bootstrap.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/footer.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/common.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/common.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/prism.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/sweetalert.min.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/jquery.canvasResize.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:34:58] "GET /static/canvasResi

70


...

41


192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /set/new HTTP/1.1" 200 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/bootstrap.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/footer.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/common.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/common.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/prism.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/sweetalert.min.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/jquery.canvasResize.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/canvasResize.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:39] "GET /static/jquery.ex

new


192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /set/29 HTTP/1.1" 200 -
192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /static/bootstrap.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /static/footer.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /static/common.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /static/common.css HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /static/prism.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /static/sweetalert.min.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:40] "GET /static/jquery.canvasResize.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:41] "GET /static/canvasResize.js HTTP/1.1" 304 -
192.168.70.205 - - [13/Dec/2022 21:35:41] "GET /static/jquery.exi

29
